In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext
sc = SparkContext("local", "Cache")

22/03/02 20:28:40 WARN Utils: Your hostname, ubuntu-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.80.128 instead (on interface ens33)
22/03/02 20:28:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/03/02 20:28:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/02 20:28:42 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
from pyspark import StorageLevel

In [11]:
def to_ascii(text):
    import re
    output = re.sub(r"[^a-zA-Z0-9 ]", "", text)
    #print(output)
    return output

text = "prince, don't; welcome"
to_ascii(text)

'prince dont welcome'

In [14]:
wordCountRdd = sc.textFile("hdfs://localhost:9000/book-war-and-peace.txt")\
                 .map (lambda line: line.strip().lower())\
                 .map (lambda line: line.split(" "))\
                 .flatMap(lambda elements: elements)\
                 .filter (lambda word: word != "")\
                 .map (lambda word: (word, 1))\
                 .reduceByKey(lambda acc, value: acc + value)

In [17]:
wordCountRdd.cache() # , call persit internally with MEMORY_ONLY

#wordCountRdd.persist(StorageLevel.MEMORY_AND_DISK) # we have many options
#wordCountRdd.persist(StorageLevel.DISK_ONLY) # we have many options


wordCountRdd.take(10)

[('chapter', 366),
 ('i', 3209),
 ('"well,', 262),
 ('prince,', 71),
 ('so', 1576),
 ('genoa', 3),
 ('and', 21327),
 ('lucca', 2),
 ('are', 1163),
 ('now', 957)]

In [20]:
sortedRddAscending = wordCountRdd.sortBy(lambda kv: kv[1])

sortedRddAscending.take(10)

[('buonapartes.', 1),
 ('infamies', 1),
 ("'faithful", 1),
 ("slave,'", 1),
 ('you--sit', 1),
 ('news."', 1),
 ('scherer,', 1),
 ('grippe;', 1),
 ('grippe', 1),
 ('elite.', 1)]

In [21]:
print(wordCountRdd.getNumPartitions())
sortedRddDescending = wordCountRdd.sortBy(lambda kv: kv[1], ascending = False)

sortedRddDescending.take(10)

1


[('the', 34088),
 ('and', 21327),
 ('to', 16426),
 ('of', 14784),
 ('a', 10357),
 ('he', 9294),
 ('in', 8551),
 ('his', 7932),
 ('that', 7402),
 ('was', 7201)]

In [22]:
wordCountRdd.saveAsTextFile("hdfs://localhost:9000/war-and-peace-1.txt")

In [24]:
sortedRddDescending.saveAsTextFile("hdfs://localhost:9000/war-and-peace-sorted-desc")

In [25]:
sortedRddAscending.saveAsTextFile("hdfs://localhost:9000/war-and-peace-sorted-asc")